[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HUSTneeb/GoogleColabCode/blob/master/TensorFlowLearningPix2Pix.ipynb)

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import PIL
from IPython.display import clear_output


In [3]:
path_to_zip=tf.keras.utils.get_file('facades.tar.gz',
                                   cache_subdir=os.path.abspath('.'),
                                   origin='https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz',
                                   extract=True)
PATH=os.path.join(os.path.dirname(path_to_zip),'facades/')

30171136/30168306 [==============================] - 0s 0us/step


In [0]:
BUFFER_SIZE=400
BATCH_SIZE=1
IMG_WIDTH=256
IMG_HEIGHT=256

In [0]:
def load_image(image_file,is_train):
  image=tf.read_file(image_file)
  image=tf.image.decode_jpeg(image)
  
  w=tf.shape(image)[1]
  
  w=w//2
  real_image=image[:,:w,:]
  input_image=image[:,w:,:]
  
  input_image=tf.cast(input_image,tf.float32)
  real_image=tf.cast(real_image,tf.float32)
  
  if is_train:
    input_image=tf.image.resize_images(input_image,[286,286],
                                      align_corners=True,
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image=tf.image.resize_images(real_image,[286,286],
                                     align_corners=True,
                                     method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    stacked_image=tf.stack([input_image,real_image],axis=0)
    cropped_image=tf.random_crop(stacked_image,size=[2,IMG_HEIGHT,IMG_WIDTH,3])
    input_image,real_image=cropped_image[0],cropped_image[1]
    
    if np.random.random()>0.5:
      input_image=tf.image.flip_left_right(input_image)
      real_image=tf.image.flip_left_right(real_image)
    else:
      input_image=tf.image.resize_images(input_image,size=[IMG_HEIGHT,IMG_WIDTH],
                                        align_corners=True,method=2)
      real_image=tf.image.resize_images(real_image,size=[IMG_HEIGHT,IMG_WIDTH],
                                       align_corners=True,method=2)
      
  input_image=(input_image/127.5)-1
  real_image=(real_image/127.5)-1
  
  return input_image,real_image
  

In [0]:
train_dataset=tf.data.Dataset.list_files(PATH+'train/*.jpg')
train_dataset=train_dataset.shuffle(BUFFER_SIZE)
train_dataset=train_dataset.map(lambda x:load_image(x,True))
train_dataset=train_dataset.batch(1)

test_dataset=tf.data.Dataset.list_files(PATH+'test/*.jpg')
test_dataset=test_dataset.map(lambda x:load_image(x,False))
test_dataset=test_dataset.batch(1)

In [0]:
OUTPUT_CHANNELS=3

class Downsample(tf.keras.Model):
  def __init__(self,filters,size,apply_batchnorm=True):
    super(Downsample,self).__init__()
    self.apply_batchnorm=apply_batchnorm
    initializer=tf.random_normal_initializer(0.,0.02)
    
    self.conv1=tf.keras.layers.Conv2D(filters,
                                     (size,size),
                                     strides=2,
                                     padding='same',
                                     kernel_initializer=initializer,
                                     use_bias=False)
    
    if self.apply_batchnorm:
      self.batchnorm=tf.keras.layers.BatchNormalization()
      
  def call(self,x,training)